# Tool Agent

In [1]:
!pip install python-dotenv langchain-community wikipedia


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
model_path = os.getenv("MODEL_PATH")

In [3]:
import logging

from app.llm.llamacpp.service import LlamaCppService

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info(f"Model path: {model_path}")
llm = LlamaCppService(model_path=model_path, n_gpu_layers=-1)

INFO:__main__:Model path: D:\llm-models\NousResearch\Hermes-2-Pro-Llama-3-8B-GGUF\Hermes-2-Pro-Llama-3-8B-Q8_0.gguf
llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from D:\llm-models\NousResearch\Hermes-2-Pro-Llama-3-8B-GGUF\Hermes-2-Pro-Llama-3-8B-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Hermes-2-Pro-Llama-3-8B
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32        

In [8]:
from app.tools.langchain.wikipedia_query_run import LangchainWikipediaQueryRun
from app.agents.react.agent import ReActAgentFactory

tools = [LangchainWikipediaQueryRun(top_k_results=3, doc_content_chars_max=2048)]
react_agent = ReActAgentFactory.start(llm=llm, tools=tools, step_limit=6)

In [10]:
from app.agents.messages import Query

future = react_agent.ask(Query(goal="What year did the iphone come out?"))
response = future.get(timeout=120)
print(f"Total tokens: {sum(step.token_usage.total_tokens for step in response.metadata['steps'])}")
print(response.final_output)

future = react_agent.ask(Query(goal="What other models exist?"))
response = future.get(timeout=120)
print(f"Total tokens: {sum(step.token_usage.total_tokens for step in response.metadata['steps'])}")
print(response.final_output)

Total tokens: 10526
The iPhone was first released on June 29, 2007. This information was obtained by using the "wikipedia_query_run" tool with the query "iPhone release date," which provided a reliable source of information from Wikipedia's knowledge base.
Total tokens: 14053
The other models that exist are several iPhone generations and series, including:

1. iPhone SE (2020)
2. iPhone XR
3. iPhone XS Max
4. iPhone 12 Pro

Each model has its unique features, specifications, and price range. By using the "wikipedia_query_run" tool with the query "iPhone models," you can retrieve a comprehensive list of all iPhone models from Wikipedia's knowledge base, which provides an overview of the different models available and their key features.


In [6]:
react_agent.stop()

True